In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras import regularizers
from keras.callbacks import ModelCheckpoint

import numpy as np

In [3]:
num_classes = 10
num_predictions = 20
batch_size = 64

(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)




# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_val=x_val.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_val=(x_val-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)


x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [4]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [5]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [6]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training


opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=150,\
                    verbose=1,validation_data=(x_val,y_val),callbacks=[LearningRateScheduler(lr_schedule),mc])

#save to disk
model_json = model.to_json()
with open('best_model.json', 'w') as json_file:
    json_file.write(model_json)



Epoch 1/150
625/625 [==============================] - 17s 27ms/step - loss: 2.0230 - acc: 0.3960 - val_loss: 1.5342 - val_acc: 0.5100

Epoch 00001: val_acc improved from -inf to 0.51000, saving model to best_model.h5
Epoch 2/150
625/625 [==============================] - 15s 24ms/step - loss: 1.3902 - acc: 0.5579 - val_loss: 1.1401 - val_acc: 0.6501

Epoch 00002: val_acc improved from 0.51000 to 0.65010, saving model to best_model.h5
Epoch 3/150
625/625 [==============================] - 15s 24ms/step - loss: 1.1580 - acc: 0.6301 - val_loss: 0.9872 - val_acc: 0.6962

Epoch 00003: val_acc improved from 0.65010 to 0.69620, saving model to best_model.h5
Epoch 4/150
625/625 [==============================] - 15s 24ms/step - loss: 1.0479 - acc: 0.6664 - val_loss: 0.9983 - val_acc: 0.7076

Epoch 00004: val_acc improved from 0.69620 to 0.70760, saving model to best_model.h5
Epoch 5/150
625/625 [==============================] - 15s 24ms/step - loss: 0.9735 - acc: 0.6929 - val_loss: 0.8024 - 


Epoch 00041: val_acc did not improve from 0.85810
Epoch 42/150
625/625 [==============================] - 15s 24ms/step - loss: 0.6194 - acc: 0.8364 - val_loss: 0.5986 - val_acc: 0.8487

Epoch 00042: val_acc did not improve from 0.85810
Epoch 43/150
625/625 [==============================] - 15s 24ms/step - loss: 0.6170 - acc: 0.8381 - val_loss: 0.6328 - val_acc: 0.8425

Epoch 00043: val_acc did not improve from 0.85810
Epoch 44/150
625/625 [==============================] - 15s 24ms/step - loss: 0.6112 - acc: 0.8402 - val_loss: 0.7008 - val_acc: 0.8241

Epoch 00044: val_acc did not improve from 0.85810
Epoch 45/150
625/625 [==============================] - 15s 24ms/step - loss: 0.6111 - acc: 0.8404 - val_loss: 0.5738 - val_acc: 0.8600

Epoch 00045: val_acc improved from 0.85810 to 0.86000, saving model to best_model.h5
Epoch 46/150
625/625 [==============================] - 15s 24ms/step - loss: 0.6026 - acc: 0.8437 - val_loss: 0.5969 - val_acc: 0.8512

Epoch 00046: val_acc did not 

625/625 [==============================] - 15s 24ms/step - loss: 0.4838 - acc: 0.8810 - val_loss: 0.5165 - val_acc: 0.8721

Epoch 00084: val_acc did not improve from 0.88040
Epoch 85/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4822 - acc: 0.8805 - val_loss: 0.5610 - val_acc: 0.8608

Epoch 00085: val_acc did not improve from 0.88040
Epoch 86/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4788 - acc: 0.8795 - val_loss: 0.5610 - val_acc: 0.8638

Epoch 00086: val_acc did not improve from 0.88040
Epoch 87/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4753 - acc: 0.8794 - val_loss: 0.4924 - val_acc: 0.8807

Epoch 00087: val_acc improved from 0.88040 to 0.88070, saving model to best_model.h5
Epoch 88/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4770 - acc: 0.8818 - val_loss: 0.5068 - val_acc: 0.8775

Epoch 00088: val_acc did not improve from 0.88070
Epoch 89/150
625/625 [=====================

625/625 [==============================] - 15s 24ms/step - loss: 0.4379 - acc: 0.8877 - val_loss: 0.5052 - val_acc: 0.8751

Epoch 00127: val_acc did not improve from 0.88840
Epoch 128/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4403 - acc: 0.8877 - val_loss: 0.4523 - val_acc: 0.8885

Epoch 00128: val_acc improved from 0.88840 to 0.88850, saving model to best_model.h5
Epoch 129/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4326 - acc: 0.8901 - val_loss: 0.5476 - val_acc: 0.8663

Epoch 00129: val_acc did not improve from 0.88850
Epoch 130/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4405 - acc: 0.8875 - val_loss: 0.5307 - val_acc: 0.8698

Epoch 00130: val_acc did not improve from 0.88850
Epoch 131/150
625/625 [==============================] - 15s 24ms/step - loss: 0.4371 - acc: 0.8873 - val_loss: 0.4992 - val_acc: 0.8762

Epoch 00131: val_acc did not improve from 0.88850
Epoch 132/150
625/625 [================

In [7]:
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


10000/10000 [==============================] - 1s 64us/step

Test result: 87.890 loss: 0.483


In [8]:
model.save('final.h5')


In [9]:
del model
from keras.models import load_model
model= load_model('best_model.h5')
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 1s 81us/step

Test result: 88.320 loss: 0.470
